In [1]:
import pandas as pd 
import numpy as np 
from tqdm import tqdm
import math
import gc

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train=pd.read_csv("../data/Train.csv")
#test=pd.read_csv("./data/Test.csv")
meta=pd.read_csv("../data/airqo_metadata.csv")

In [3]:
train.head()


,ID,location,temp,precip,rel_humidity,wind_dir,wind_spd,atmos_press,target
0,ID_train_0,C,"nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...",45.126304
1,ID_train_1,D,"22.53333333,21.71666667,20.83333333,20.9833333...","0.102,0.0,0.0,0.0,0.0,0.0,0.0,0.034,0.017,0.01...","0.744583333,0.808083333,0.911166667,0.91633333...","281.6643101,89.15629262,81.96853891,291.018632...","2.3775,1.126666667,0.700833333,0.3416666670000...","90.32,90.3775,90.44083333,90.4725,90.45416667,...",79.131702
2,ID_train_10,A,"28.975,27.95,29.6,26.425,22.09166667,21.775,22...","0.0,0.0,0.0,0.102,0.136,0.0,0.0,2.16,1.276,0.0...","0.573333333,0.597166667,0.5668333329999999,0.6...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","88.55166667,88.46416667,88.31916667,88.24,88.2...",32.661304
3,ID_train_100,A,"22.96666667,24.26666667,25.275,25.625,25.86666...","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.77,3.012,1.0...","0.8430833329999999,0.79025,0.7375,0.728,0.7049...","300.0850574,293.6769595,294.5174647,301.921416...","1.446666667,1.1925,1.324166667,1.5441666669999...","88.615,88.53083333,88.4,88.27166667,88.2075,88...",53.850238
4,ID_train_1000,A,"21.875,21.575,21.525,21.43333333,20.50833333,1...","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.8564166670000001,0.874916667,0.879833333,0.8...","21.83997432,17.05405341,89.26406044,123.585424...","0.1975,0.244166667,0.411666667,0.56,0.5775,0.4...","88.55666667,88.64083333,88.65833333,88.6475,88...",177.418750


In [4]:
# covert features  from string to List of values 
def replace_nan(x):
    if x==" ":
        return np.nan
    else :
        return float(x)
        
features=["temp","precip","rel_humidity","wind_dir","wind_spd","atmos_press"]
for feature in features : 
    train[feature]=train[feature].apply(lambda x: [ replace_nan(X) for X in x.replace("nan"," ").split(",")])
    #test[feature]=test[feature].apply(lambda x: [ replace_nan(X)  for X in x.replace("nan"," ").split(",")])    

count nan values

In [5]:
list_columns = ['temp', 'precip', 'rel_humidity', 'wind_dir', 'wind_spd', 'atmos_press']
def count_nan(row):
    counter = 0
    if isinstance(row, type(list())):
        for x in row:
            if np.isnan(x):
                counter += 1
        return counter 
    else: return np.nan
    
for col in list_columns:
    train['nan_' + str(col)] = round(train[col].apply(count_nan))
data_nan = train.drop(list_columns, axis=1)

data_nan['sum'] = round(data_nan.sum(axis=1, numeric_only=True)/726*100,2)
data_nan.shape

(15539, 10)

In [6]:
data_wo_outlier = train.join(data_nan['sum'])[(train.join(data_nan['sum'])['sum']<50)]
data_wo_outlier.shape

(14536, 16)

In [7]:
data_wo_outlier.head(2)

,ID,location,temp,precip,rel_humidity,wind_dir,wind_spd,atmos_press,target,nan_temp,nan_precip,nan_rel_humidity,nan_wind_dir,nan_wind_spd,nan_atmos_press,sum
1,ID_train_1,D,"[22.53333333, 21.71666667, 20.83333333, 20.983...","[0.102, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.034, 0...","[0.744583333, 0.808083333, 0.911166667, 0.9163...","[281.6643101, 89.15629262, 81.96853891, 291.01...","[2.3775, 1.126666667, 0.700833333, 0.341666667...","[90.32, 90.3775, 90.44083333, 90.4725, 90.4541...",79.131702,0,0,0,0,0,0,10.90
2,ID_train_10,A,"[28.975, 27.95, 29.6, 26.425, 22.09166667, 21....","[0.0, 0.0, 0.0, 0.102, 0.136, 0.0, 0.0, 2.16, ...","[0.573333333, 0.597166667, 0.5668333329999999,...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[88.55166667, 88.46416667, 88.31916667, 88.24,...",32.661304,1,1,1,46,46,1,17.72


fill nan values with mean

In [8]:
# removing the nan values and show the remain values
def mean_without_nan(row):
    mean = np.nanmean(row)
    new_row = []
    for x in row: 
        if np.isnan(x): new_row.append(mean)
        else: new_row.append(x)
    return new_row

In [9]:
features = data_wo_outlier.columns[2:8]
features
nan_featuers = ["nan_"+ str(feature) for feature in features]
nan_featuers

['nan_temp',
 'nan_precip',
 'nan_rel_humidity',
 'nan_wind_dir',
 'nan_wind_spd',
 'nan_atmos_press']

In [10]:
# fill nan
for feat in features:
    data_wo_outlier["new_" + str(feat)] = data_wo_outlier[feat].apply(mean_without_nan)




In [11]:
features_to_drop = features #.extend(nan_featuers,['sum'])
data_process = data_wo_outlier.drop(features, axis=1)
data_process.drop(nan_featuers, axis=1, inplace=True)
data_process.drop(['sum'], axis=1, inplace=True)


In [12]:
data_process
data_process.head()

,ID,location,target,new_temp,new_precip,new_rel_humidity,new_wind_dir,new_wind_spd,new_atmos_press
1,ID_train_1,D,79.131702,"[22.53333333, 21.71666667, 20.83333333, 20.983...","[0.102, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.034, 0...","[0.744583333, 0.808083333, 0.911166667, 0.9163...","[281.6643101, 89.15629262, 81.96853891, 291.01...","[2.3775, 1.126666667, 0.700833333, 0.341666667...","[90.32, 90.3775, 90.44083333, 90.4725, 90.4541..."
2,ID_train_10,A,32.661304,"[28.975, 27.95, 29.6, 26.425, 22.09166667, 21....","[0.0, 0.0, 0.0, 0.102, 0.136, 0.0, 0.0, 2.16, ...","[0.573333333, 0.597166667, 0.5668333329999999,...","[201.37321071165334, 201.37321071165334, 201.3...","[1.0386555556, 1.0386555556, 1.0386555556, 1.0...","[88.55166667, 88.46416667, 88.31916667, 88.24,..."
3,ID_train_100,A,53.850238,"[22.96666667, 24.26666667, 25.275, 25.625, 25....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.77,...","[0.8430833329999999, 0.79025, 0.7375, 0.728, 0...","[300.0850574, 293.6769595, 294.5174647, 301.92...","[1.446666667, 1.1925, 1.324166667, 1.544166666...","[88.615, 88.53083333, 88.4, 88.27166667, 88.20..."
5,ID_train_10000,E,17.005000,"[26.225, 26.25, 26.95833333, 27.925, 28.416666...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.7566666670000001, 0.7090833329999999, 0.632...","[168.29624559847116, 104.3737208, 161.28512859...","[1.1332211538846155, 1.4858333330000002, 1.090...","[88.56583333, 88.55, 88.50916667, 88.44083333,..."
6,ID_train_10001,C,53.100000,"[21.86666667, 21.38333333, 20.75, 20.74166667,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.548333333, 0.571666667, 0.589166667, 0.57, ...","[334.0827161, 332.4037603, 328.58290530000005,...","[1.1525, 1.055, 1.11, 1.385833333, 1.2, 1.2508...","[87.73666667, 87.74416667, 87.7475, 87.78, 87...."


- create new dataframes for each feature and observation

In [13]:
# name of dataframes:
df_names = ["df_" + str(feat) for feat in features]
df_names

['df_temp',
 'df_precip',
 'df_rel_humidity',
 'df_wind_dir',
 'df_wind_spd',
 'df_atmos_press']

In [14]:
features
for f in features: 
    exec(f'df_{f} = pd.DataFrame(data_process["new_{f}"])')
    

unpack all values: 

In [15]:
df_atmos_press

,new_atmos_press
1,"[90.32, 90.3775, 90.44083333, 90.4725, 90.4541..."
2,"[88.55166667, 88.46416667, 88.31916667, 88.24,..."
3,"[88.615, 88.53083333, 88.4, 88.27166667, 88.20..."
5,"[88.56583333, 88.55, 88.50916667, 88.44083333,..."
6,"[87.73666667, 87.74416667, 87.7475, 87.78, 87...."
...,...
15534,"[88.27666667, 88.24333333, 88.18166667, 88.175..."
15535,"[88.43666667, 88.34583333, 88.26083333, 88.174..."
15536,"[90.61583333, 90.44916667, 90.32916667, 90.27,..."
15537,"[90.31416667, 90.23916667, 90.20166667, 90.222..."


In [16]:
df_names

['df_temp',
 'df_precip',
 'df_rel_humidity',
 'df_wind_dir',
 'df_wind_spd',
 'df_atmos_press']

In [ ]:
for dframe in df_names:
    df = locals()[dframe]
    name = df.columns[0]
    for x in range(121): # 121 messdaten pro punkt 
        df[name+ str(x)] = df[name].str[x]
        #df.drop(name, axis=1, inplace=True)
    

In [21]:
for dframe in df_names:
    df = locals()[dframe]
    name = df.columns[0]
    df.drop(name, axis=1, inplace=True)

add meta information

In [19]:
meta.head()

,Unnamed: 0,location,loc_altitude,km2,aspect,dist_motorway,dist_trunk,dist_primary,dist_secondary,dist_tertiary,dist_unclassified,dist_residential,popn,hh,hh_cook_charcoal,hh_cook_firewood,hh_burn_waste
0,0,A,1122.4,1.9,194.0,NaN,NaN,14.695789,343.595039,575.917422,330.609776,254.307415,4763,809,508,43,142
1,1,B,1155.4,5.4,219.8,NaN,528.078476,2172.680462,1144.376412,531.103271,65.142004,1.042809,22243,5735,116,1144,239
2,2,C,1178.3,8.5,168.7,NaN,32.885520,4794.704552,12.730489,72.459340,406.967815,13.931350,97895,26873,21316,751,9835
3,3,D,980.8,0.8,90.0,NaN,NaN,265.896472,NaN,339.693908,76.024389,11.601442,3596,663,7,651,99
4,4,E,1186.5,1.6,121.0,NaN,850.423131,NaN,1738.872942,137.722087,449.591885,3.836589,6064,1297,985,26,43


input nan dist_trunk, dist_primary and dist_secondary with knn imputer

In [26]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=2)
meta_imp = imputer.fit_transform(meta.drop(['location'], axis=1))

In [52]:
meta_imp = pd.DataFrame(meta_imp)
meta_imp.columns = [col for col in meta.columns[1:] if col != 'dist_motorway']
meta_imp

,location,loc_altitude,km2,aspect,dist_trunk,dist_primary,dist_secondary,dist_tertiary,dist_unclassified,dist_residential,popn,hh,hh_cook_charcoal,hh_cook_firewood,hh_burn_waste
0,0.0,1122.4,1.9,194.0,689.250804,14.695789,343.595039,575.917422,330.609776,254.307415,4763.0,809.0,508.0,43.0,142.0
1,1.0,1155.4,5.4,219.8,528.078476,2172.680462,1144.376412,531.103271,65.142004,1.042809,22243.0,5735.0,116.0,1144.0,239.0
2,2.0,1178.3,8.5,168.7,32.885520,4794.704552,12.730489,72.459340,406.967815,13.931350,97895.0,26873.0,21316.0,751.0,9835.0
3,3.0,980.8,0.8,90.0,689.250804,265.896472,1041.233990,339.693908,76.024389,11.601442,3596.0,663.0,7.0,651.0,99.0
4,4.0,1186.5,1.6,121.0,850.423131,140.296131,1738.872942,137.722087,449.591885,3.836589,6064.0,1297.0,985.0,26.0,43.0


In [57]:
meta_imp['location'] = meta.location

In [58]:
meta_imp

,location,loc_altitude,km2,aspect,dist_trunk,dist_primary,dist_secondary,dist_tertiary,dist_unclassified,dist_residential,popn,hh,hh_cook_charcoal,hh_cook_firewood,hh_burn_waste
0,A,1122.4,1.9,194.0,689.250804,14.695789,343.595039,575.917422,330.609776,254.307415,4763.0,809.0,508.0,43.0,142.0
1,B,1155.4,5.4,219.8,528.078476,2172.680462,1144.376412,531.103271,65.142004,1.042809,22243.0,5735.0,116.0,1144.0,239.0
2,C,1178.3,8.5,168.7,32.885520,4794.704552,12.730489,72.459340,406.967815,13.931350,97895.0,26873.0,21316.0,751.0,9835.0
3,D,980.8,0.8,90.0,689.250804,265.896472,1041.233990,339.693908,76.024389,11.601442,3596.0,663.0,7.0,651.0,99.0
4,E,1186.5,1.6,121.0,850.423131,140.296131,1738.872942,137.722087,449.591885,3.836589,6064.0,1297.0,985.0,26.0,43.0


add the meta data into each dataframe according to the location

- add meta daten to the df


In [73]:
for dframe in df_names:
    df = locals()[dframe]
    
    name = df.columns[0]
    df_1 = df.join(data_process["location"])
    exec(f'f_{name} = pd.merge(df_1, meta_imp, how="left", on="location")')

In [75]:
list_features = []
for dframe in df_names:
    df = locals()[dframe]
    name = df.columns[0]
    list_features.append("f_"+str(name))

In [76]:
list_features

['f_new_temp0',
 'f_new_precip0',
 'f_new_rel_humidity0',
 'f_new_wind_dir0',
 'f_new_wind_spd0',
 'f_new_atmos_press0']

In [77]:
data_process.target

1         79.131702
2         32.661304
3         53.850238
5         17.005000
6         53.100000
            ...    
15534     44.850286
15535     24.330455
15536     38.972128
15537     41.720952
15538    127.983333
Name: target, Length: 14536, dtype: float64